# ML Pipeline Preparation

### 1. Import libraries and load data from database.

In [79]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import nltk
import joblib

In [69]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_score, recall_score, f1_score,accuracy_score
from sklearn.metrics import classification_report,confusion_matrix
import pickle

In [49]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('df',con=engine)
X = df['message']
y = df.drop(['id','message','original','genre'],axis=1)
category_names = y.columns

### 2. Tokenization function to process text data

In [29]:
def tokenize(text):
    word = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_token = []
    for token in word:
        lem_tok = lemmatizer.lemmatize(token).lower().strip()
        clean_token.append(lem_tok)
        
    return clean_token

### 3. Building a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset.

In [111]:
pipeline = Pipeline([

        ('vect',CountVectorizer(tokenizer=tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf',MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [112]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [113]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

In [114]:
pred = pipeline.predict(X_test)

### 5. Testing model

In [115]:
performances = []
for i in range(len(category_names)):
        performances.append([f1_score(y_test.iloc[:, i].values, pred[:, i], average='micro'),
                             precision_score(y_test.iloc[:, i].values, pred[:, i], average='micro'),
                             recall_score(y_test.iloc[:, i].values, pred[:, i], average='micro')])

In [116]:
performances = pd.DataFrame(performances, columns=['f1 score', 'precision', 'recall'],
                                index = category_names)

In [117]:
performances

,f1 score,precision,recall
related,0.793770,0.793770,0.793770
request,0.894342,0.894342,0.894342
offer,0.995423,0.995423,0.995423
aid_related,0.769866,0.769866,0.769866
medical_help,0.921933,0.921933,0.921933
medical_products,0.948506,0.948506,0.948506
search_and_rescue,0.975842,0.975842,0.975842
security,0.981310,0.981310,0.981310
military,0.969358,0.969358,0.969358
child_alone,1.000000,1.000000,1.000000


### 6. Improving model
Using grid search to find better parameters. 

In [124]:
parameters = {
              'clf__estimator__max_features':['sqrt', 0.5],
              'clf__estimator__n_estimators':[50, 100]}

cv = GridSearchCV(estimator=pipeline, param_grid = parameters, cv = 5)

In [126]:
cv.fit(X_train,y_train)

In [129]:
cv.best_params_

In [128]:
joblib.dump(pipeline,'model.pkl')

['model.pkl']

In [137]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26211,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26214,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
